In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers torch sklearn pandas

  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [5]:
import transformers
import os
from transformers import BertTokenizer, BertModel, AdamW, BertForSequenceClassification
import torch
import pickle
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, TensorDataset, random_split, Dataset
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, f1_score, precision_score, recall_score
from sklearn.preprocessing import LabelEncoder, MultiLabelBinarizer
import torch.nn as nn
from torch.nn import functional


In [9]:
# Load dataset
try:
    df = pd.read_csv('/content/drive/MyDrive/train_title_abstract_keywords.csv', encoding='utf-8')
except UnicodeDecodeError:
    df = pd.read_csv('/content/drive/MyDrive/train_title_abstract_keywords.csv', encoding='latin1')
print(df.columns)

print("Length of the DataFrame:", len(df))
df = df.head(10000)

# Preprocess and prepare data
df['combined_text'] = df['Title'] + " " + df['Abstract']

# Preprocess and prepare data
tokenizer = BertTokenizer.from_pretrained('allenai/scibert_scivocab_uncased')
max_length = 128

def tokenize_and_encode(texts):
    input_ids = []
    attention_masks = []

    for text in texts:
        encoded = tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=max_length,
            padding='max_length',
            return_attention_mask=True,
            truncation=True
        )
        input_ids.append(encoded['input_ids'])
        attention_masks.append(encoded['attention_mask'])

    return torch.tensor(input_ids), torch.tensor(attention_masks)

input_ids, attention_masks = tokenize_and_encode(df['combined_text'])



Index(['id', 'Title', 'Subtitle', 'Abstract', 'Keywords'], dtype='object')
Length of the DataFrame: 148928


vocab.txt:   0%|          | 0.00/228k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

In [6]:
tokenized_data_path = "/content/drive/MyDrive/scibert_tokenized_data.pkl"

# Load the data
with open(tokenized_data_path, 'rb') as file:
    tokenized_data = pickle.load(file)

input_ids, attention_masks = tokenized_data

In [7]:
torch.cuda.empty_cache()

In [10]:
# MultiLabelBinarizer for labels
keywords = df['Keywords'].str.split(',')
# Binarize labels
mlb = MultiLabelBinarizer()
labels = mlb.fit_transform(keywords)
# Convert labels to tensor
labels = torch.tensor(labels)


# Create Dataset
class TextDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

dataset = TextDataset({'input_ids': input_ids, 'attention_mask': attention_masks}, labels)

# Split dataset into training and validation
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Create DataLoaders
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)


class SciBertForMultiLabelClassification(nn.Module):
    def __init__(self, num_labels):
        super().__init__()
        self.bert = BertModel.from_pretrained("allenai/scibert_scivocab_uncased")
        self.classifier = nn.Linear(self.bert.config.hidden_size, num_labels)
        self.sigmoid = nn.Sigmoid()

    def forward(self, input_ids, attention_mask):
        output = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = output.pooler_output
        logits = self.classifier(pooled_output)
        return self.sigmoid(logits)

num_labels = len(mlb.classes_)
model = SciBertForMultiLabelClassification(num_labels=num_labels)
model.cuda()



pytorch_model.bin:   0%|          | 0.00/442M [00:00<?, ?B/s]

SciBertForMultiLabelClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(31090, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1

In [11]:
 # Save the tokenized data
 tokenized_data_path = "/content/drive/MyDrive/scibert_tokenized_data_updated.pkl"
 tokenized_data = (input_ids, attention_masks)
 with open(tokenized_data_path, 'wb') as file:
  pickle.dump(tokenized_data, file)


In [14]:
optimizer = AdamW(model.parameters(), lr=5e-5)
loss_fn = nn.BCELoss()

# Training loop
for epoch in range(10):
    model.train()
    total_loss = 0
    for batch in train_loader:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].cuda()
        attention_mask = batch['attention_mask'].cuda()
        labels = batch['labels'].float().cuda()
        outputs = model(input_ids, attention_mask)
        loss = loss_fn(outputs, labels)
        total_loss += loss.item()
        loss.backward()
        optimizer.step()

    avg_train_loss = total_loss / len(train_loader)
    print(f"Training loss for epoch {epoch}: {avg_train_loss}")

    # Validation step
    model.eval()
    total_eval_loss = 0
    all_preds = []
    all_label_ids = []
    for batch in val_loader:
        input_ids = batch['input_ids'].cuda()
        attention_mask = batch['attention_mask'].cuda()
        labels = batch['labels'].float().cuda()
        outputs = model(input_ids, attention_mask)
        loss = loss_fn(outputs, labels)
        total_eval_loss += loss.item()

        # Convert outputs to binary predictions (0 or 1)
        preds = torch.sigmoid(outputs).cpu().detach().numpy()
        preds = np.where(preds > 0.5, 1, 0)
        all_preds.extend(preds)
        all_label_ids.extend(labels.cpu().detach().numpy())

    avg_val_loss = total_eval_loss / len(val_loader)
    val_f1_score = f1_score(all_label_ids, all_preds, average='micro')
    print(f"Validation loss for epoch {epoch}: {avg_val_loss}")
    print(f"Validation F1 Score for epoch {epoch}: {val_f1_score}")


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
<ipython-input-10-92259c2f99dc>:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-10-92259c2f99dc>:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item['labels'] = torch.tensor(self.labels[idx])


Training loss for epoch 0: 0.007431948192417622
Validation loss for epoch 0: 0.006563827279023826
Validation F1 Score for epoch 0: 0.0002969581540221803


<ipython-input-10-92259c2f99dc>:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-10-92259c2f99dc>:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item['labels'] = torch.tensor(self.labels[idx])


Training loss for epoch 1: 0.005795308038592338
Validation loss for epoch 1: 0.00506509562546853
Validation F1 Score for epoch 1: 0.0002969581540221803


<ipython-input-10-92259c2f99dc>:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-10-92259c2f99dc>:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item['labels'] = torch.tensor(self.labels[idx])


...

Training loss for epoch 7: 0.002327753024175763
Validation loss for epoch 7: 0.002229741534392815
Validation F1 Score for epoch 7: 0.0002969581540221803


<ipython-input-10-92259c2f99dc>:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-10-92259c2f99dc>:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item['labels'] = torch.tensor(self.labels[idx])


Training loss for epoch 8: 0.0021774264872074125
Validation loss for epoch 8: 0.0020990365082980134
Validation F1 Score for epoch 8: 0.0002969581540221803


<ipython-input-10-92259c2f99dc>:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-10-92259c2f99dc>:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item['labels'] = torch.tensor(self.labels[idx])


Training loss for epoch 9: 0.002060961346141994
Validation loss for epoch 9: 0.0019968626147601753
Validation F1 Score for epoch 9: 0.0002969581540221803


In [21]:
import pandas as pd
from transformers import BertTokenizer

# Load dataset
test_df = pd.read_csv("/content/drive/MyDrive/test_title_abstract_keywords.csv")
test_df = test_df.head(10000)
abstracts = test_df['Abstract']

# Tokenize abstracts
tokenizer = BertTokenizer.from_pretrained('allenai/scibert_scivocab_uncased')
max_length = 128

def tokenize_and_encode(texts):
    input_ids = []
    attention_masks = []
    for text in texts:
        encoded = tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=max_length,
            padding='max_length',
            return_attention_mask=True,
            truncation=True
        )
        input_ids.append(encoded['input_ids'])
        attention_masks.append(encoded['attention_mask'])
    return torch.tensor(input_ids), torch.tensor(attention_masks)

input_ids, attention_masks = tokenize_and_encode(abstracts)


In [22]:
from torch.utils.data import TensorDataset, DataLoader

batch_size = 64
test_data = TensorDataset(input_ids, attention_masks)
test_loader = DataLoader(test_data, batch_size=batch_size)


In [27]:
model.eval()
predictions = []

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch[0].cuda()
        attention_mask = batch[1].cuda()
        outputs = model(input_ids, attention_mask)
        preds = torch.sigmoid(outputs).cpu().detach().numpy()
        predictions.extend(preds)

print("Predictions shape:", np.array(predictions).shape)
print("Number of classes in mlb:", len(mlb.classes_))

predicted_keywords = []
for pred in predictions:
    rounded_preds = np.round(pred).reshape(1, -1)
    if rounded_preds.shape[1] == len(mlb.classes_):
        keywords = mlb.inverse_transform(rounded_preds)
        predicted_keywords.append(keywords[0])
    else:
        predicted_keywords.append([])




Predictions shape: (10000, 28855)
Number of classes in mlb: 28855


In [30]:
# Print abstracts and their predicted keywords for the first 10 entries
for i in range(3):
    abstract = test_df.iloc[i]['Abstract']
    keywords = predicted_keywords[i]
    print(f"Abstract {i+1}: {abstract}")
    print(f"Predicted Keywords: {', '.join(keywords)}\n")


Abstract 1: Massive Open Online Courses (MOOCs) can deliver advanced course material at planetary scale, combining internet-based video content delivery, and cloud-based assignments. From March to May 2013, I taught the world's first EDA MOOC, entitled VLSI CAD: Logic to Layout, based on roughly 20 years of experience teaching electronic design automation in a conventional face-to-face classroom setting. Over 17,000 participants registered for this MOOC. This paper summarizes my experience with teaching EDA at planetary scale: how we covered ASIC synthesis, verification, layout, and timing; how we built cloud resources to enable students to experiment with open-source tools; how we designed software projects and deployed cloud-based auto-graders to support realistic EDA tool projects. The paper also discusses what MOOCs could mean to the dynamism of the EDA community.
Predicted Keywords:  "arithmetic's",  "banc d'&eacute;valuation ihm",  "children's book apps",  "children's speech reco

In [33]:
!pip install wordcloud


In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# Concatenate all keywords into a single string
all_keywords = [keyword for sublist in predicted_keywords for keyword in sublist]
text = ' '.join(all_keywords)

# Generate a word cloud image
wordcloud = WordCloud(background_color='white', max_words=200, contour_width=3, contour_color='steelblue').generate(text)

# Display the generated image
plt.figure(figsize=(10, 8))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()


In [ ]:
from collections import Counter
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# Flatten the list of predicted keywords and count the frequencies
all_keywords = [keyword for sublist in predicted_keywords for keyword in sublist]
keyword_counts = Counter(all_keywords)

# Select the top 100 keywords
top_keywords = dict(keyword_counts.most_common(100))
wordcloud = WordCloud(background_color='white', max_words=100).generate_from_frequencies(top_keywords)

# Display the generated image
plt.figure(figsize=(10, 8))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()
